In [4]:
import samna
import time

print(samna.__version__)

d= samna.device.get_unopened_devices()
dk = samna.device.open_device(d[0])

pm = dk.get_power_monitor()
buf = samna.graph.sink_from(pm.get_source_node())

sw = dk.get_stop_watch()
# has to enable stopwatch to use auto power measurement
sw.start(True)
pm.start_auto_power_measurement(100) # 100Hz

print("waiting for events")
events = buf.get_n_events(1000)
pm.stop_auto_power_measurement()

counts = [0,0,0]
sums = [0,0,0]
avgs = [0,0,0]

for e in events:
    sums[e.channel] += e.value
    counts[e.channel] += 1

idx = 0
for sum, count in zip(sums, counts):
    avgs[idx] = sum/count * 1000
    idx += 1

print(avgs, " in mW")


0.39.3.0
waiting for events
[0.2370484259650864, 0.01359621424977785, 1.381604718296109]  in mW


In [7]:
import samna

# Open the device and connect to the power monitor.
board = samna.device.open_device("XyloAudio3TestBoard")
power_monitor = board.get_power_monitor()
sink_pm = samna.graph.sink_from(power_monitor.get_source_node())
stopwatch = board.get_stop_watch()

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]
print(avgs)

[0.23629428372563174, 0.01388680098112142, 1.3831259073534359]
